In this notebook vectors for e2v are computed as averaged vectors from w2v for words in emoji descriptions

In [1]:
import pickle as pk
import gensim.models as gs
import numpy as np
import random
import itertools
# Internal dependencies
import nltk
from nltk.corpus import wordnet as wn
from model import Emoji2Vec
from trainer import Trainer

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import zipfile
import os
import random
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

from tfnn.layers import *
import tfnn
import math
import mt.bleu
import sys
import numpy as np
import collections
from mt.strutils import tokenize
import time

In [57]:
w2v = gs.KeyedVectors.load_word2vec_format("../data/word2vec/GoogleNews-vectors-negative300.bin", binary=True)

In [75]:
w2v_ = {}

In [76]:
counts = {}
for k, v in w2v.vocab.items():
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    word = k.lower()
    if word in counts:
        if max(v.count, counts[word]) != counts[word]:
            w2v_[word] = w2v[k]
        counts[word] = max(v.count, counts[word])
    else:
        counts[word] = v.count
        w2v_[word] = w2v[k]

In [78]:
counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)

In [79]:
counts[:10]

[('</s>', 3000000),
 ('in', 2999999),
 ('for', 2999998),
 ('that', 2999997),
 ('is', 2999996),
 ('on', 2999995),
 ('##', 2999994),
 ('the', 2999993),
 ('with', 2999992),
 ('said', 2999991)]

In [80]:
def make_data(datafile):
    
    def normalize_word(word):
        alphabet = 'abcdefghijklmnopqrstuvwxyz'
        word = word.lower()
        try:
            while word[0] not in alphabet:
                word = word[1:]
            while word[-1] not in alphabet:
                word = word[:-1]
        except:
            return None
        if word not in w2v:
            print(word)
            return None
        return word
    
    smiles_to_words = {} 
    emojis = []
    words_from_emojis = []
    with open(datafile, 'r') as f:
        lines = f.readlines()
        for line in lines:
            try:
                em, phrase, truth = line.rstrip().split('\t')
            except:
                print(em, phrase)
                continue
            phrase = phrase.lower()
            new_words = [normalize_word(new_word) for new_word in phrase.split() if not normalize_word(new_word) is None]
            if em in smiles_to_words:
                smiles_to_words[em].extend(new_words)
            else:
                smiles_to_words[em] = new_words
            emojis.append(em)
            words_from_emojis.extend(new_words)
    for key, value in smiles_to_words.items():
        normalized_words = []
        for word in list(set(value)):
            norm_word = normalize_word(word)
            if not norm_word is None:
                normalized_words.append(norm_word)
        smiles_to_words[key] = normalized_words
    emojis = list(set(emojis))
    words_from_emojis = list(set(words_from_emojis))
    return smiles_to_words, emojis, words_from_emojis

In [81]:
smiles_to_words, emojis, words_from_emojis = make_data('../data/emojipedia_positive.txt')

bajan
globified
uzbekistan
of
liechtenstein
theatre
dogeza
and
kazakhstan
world’s
asia-australia
ideograph
eight-thirty
of
leonean
bahrani
of
to
o’clock
djibouti
ni-vanuatu\xa0flag
mansplaining
moustache
algerian
of
tokelau
cjk
ideograph
bakkureru
keycap
cjk
ideograph
lithuanian
of
of
comoran
guinea-bissau
heart-shaped
eleven-thirty
gibraltar
i-kiribati
non-potable
of
shinto
uzbek
batswana
a
gibraltarian
and
brunei
tanabata
kitts
nevis
of
macedonian
tsukimi
maldives
armenia
of
kwik-e-mart
a
tuvaluan
xa0letter
o’clock
of
three-thirty
aquarius
bullant
gambian\xa0flag
keycap
palauan
gb-sct
down-pointing
positive\xa0chart
aeroplane
turkmen
and
aruban
estonian
a
of
madagascar
curly\xa0page
bosnia
herzegovina
womens\xa0toilet
slovak\xa0flag
o’clock
abcd
lithuania
furai
tanabata
and
megafrown
of
selfie
somalian\xa0flag
barbadian
right-pointing
guamanian
of
europe-africa
acoustic\xa0guitar
liberian
blond-haired
and
salvador
right\xa0curved
suriname
of
pinocchio
belize
botswana
spiral\xa0eyes
s

In [82]:
reverse_dictionary_w2v = [k[0] for k in counts[:80000]]
dictionary_w2v = {k:i+len(emojis) for i, k in enumerate(reverse_dictionary_w2v)}
emojis_to_ind = {emojis[i]: i for i in range(len(emojis))}

In [83]:
smiles = list(smiles_to_words.keys())
embeddings_array_emoji = []
ind_to_emoji = []
emoji_to_ind = {}
def make_emoji_embeddings():
    emoji_embeddings = {}
    for i in range(len(emojis)):
        curr_words = smiles_to_words[smiles[i]]
        if len(curr_words) < 1:
            continue
        smile = smiles[i]
        
        embeddings = [0 for i in range(300)]
        num_embeddings = 0
        for word in curr_words:
            if len(word) < 3 or word in ['the', 'this', 'that', 'are']:
                continue
                
            embeddings = [embeddings[j]+w2v[word][j] for j in range(300)]
            num_embeddings += 1
        embeddings = [embeddings[j]/num_embeddings for j in range(300)]
        embeddings_array_emoji.append(embeddings)
        ind_to_emoji.append(smile)
        emoji_to_ind[smile] = len(emoji_to_ind)
        emoji_embeddings[smile] = embeddings
    return emoji_embeddings

In [84]:
embeddings = make_emoji_embeddings()

In [86]:
reverse_dictionary = []
reverse_dictionary.extend(emojis)
reverse_dictionary.extend(reverse_dictionary_w2v[:words_voc_size])

Let's check the similarity between vectors got for emojis and Google's w2v vectors for corresponding words:

In [104]:
graph = tf.Graph()
with graph.as_default():
    # Input data.
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    #embeddings_words = tf.placeholder(tf.float32, shape=[len(words_from_emojis)+len(emojis), embedding_size])
    
    with tf.device('/cpu:0'):
        embeddings = tf.Variable(np.array(full_embeddings), dtype=tf.float32)
        
        # Compute the cosine similarity between minibatch examples and all embeddings.
        norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
        normalized_embeddings = embeddings / norm
        valid_embeddings = tf.nn.embedding_lookup(
            normalized_embeddings, valid_dataset)
        similarity = tf.matmul(
            valid_embeddings, normalized_embeddings, transpose_b=True)

        # Add variable initializer.
        init = tf.global_variables_initializer()

In [110]:
def write_embeddings(embeddings):
    with open("e2v_as_average.txt", "w") as file:
        for i in range(len(emojis)):
            file.write(emojis[i])
            file.write('\t')
            for j in range(300):
                file.write(str(embeddings[i][j]))
                file.write(' ')
            file.write('\n')
    with open("w2v_as_average.txt", "w") as file:
        for i in range(words_voc_size):
            file.write(reverse_dictionary[i+len(emojis)])
            file.write('\t')
            for j in range(300):
                file.write(str(embeddings[i+len(emojis)][j]))
                file.write(' ')
            file.write('\n')

In [109]:
config = tf.ConfigProto(
    device_count = {'GPU': 1}
)
with tf.Session(graph=graph, config=config) as session:
    init.run()
    print('Initialized')

    sim = session.run(similarity)
    for i in xrange(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8  # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k + 1]
        log_str = 'Nearest to %s:' % valid_word
        for k in xrange(top_k):
            close_word = reverse_dictionary[nearest[k]]
            log_str = '%s %s,' % (log_str, close_word)
        print(log_str)
        
    final_embeddings = session.run(normalized_embeddings)
    write_embeddings(final_embeddings)

Initialized
Nearest to 4️⃣: 3️⃣, 5️⃣, 6️⃣, 7️⃣, 8️⃣, 2️⃣, 9️⃣, 1️⃣,
Nearest to 🥐: baguette, 🥖, muffin, 🍮, cappuccino, crepes, scones, sandwich,
Nearest to 🎐: jellyfish, chime, 🎏, 🔔, bells, chimes, wind, 🔕,
Nearest to 🐑: sheep, lambs, ewes, heifers, heifer, calves, cattle, cows,
Nearest to 😭: 😂, crying, sobbing, bawling, cried, weeping, 😢, 😹,
Nearest to 🍝: pasta, spaghetti, ravioli, polenta, tomato_sauce, risotto, lasagna, meatballs,
Nearest to 🐏: ramming, rammed, scuttle, push, flog, battering_ram, 🐪, shove,
Nearest to 🍽️: 🍴, fork, 🔪, dish, plate, knife, dishes, forks,
Nearest to 😏: 😼, 😉, smirking, 😙, 😊, 😚, 😀, ☺️,
Nearest to 👨‍👨‍👧‍👦: 👨‍👨‍👧‍👧, 👨‍👨‍👦‍👦, 👩‍👩‍👧‍👦, 👨‍👨‍👧, 👨‍👨‍👦, 👩‍👩‍👦‍👦, 👨‍👩‍👧‍👦, 👨‍👩‍👦‍👦,
Nearest to ⬜: ⬛, ▫️, ▪️, ◽, ◾, ◻️, ◼️, ⏹️,
Nearest to 🇱🇰: 🇳🇵, 🇪🇹, 🇿🇼, 🇱🇦, 🇵🇰, 🇲🇲, 🇰🇪, 🇲🇱,
Nearest to 🇨🇫: 🇨🇿, 🇿🇲, 🇿🇦, 🇪🇭, 🇷🇸, 🇬🇭, 🇪🇹, 🇿🇼,
Nearest to 🔩: screw, nut, screws, bolts, 🌰, nuts, ⚡, latch,
Nearest to 💑: love, ❤️, loving, 👨‍❤️‍👨, 👩‍❤️‍👩, 💔, 🏩, loved,
Nearest to ⭕: 🔴, 🔵, 🔽, 🔼, ✅, ⚪, 

Nearest to 🇨🇵: 🇨🇵, 🇳🇫, 🇫🇰, 🇵🇳, 🇸🇧, 🇰🇾, 🇫🇴, 🇲🇵,
Nearest to 🤡: 👹, 🌚, 👺, 🕴️, privacy, 🐯, clown, 🙃,
Nearest to ❕: ‼️, ⁉️, exclamation, 🆙, ❗, ❣️, 🔛, ❔,
Nearest to 🇪🇷: 🇸🇭, 🇳🇮, 🇲🇪, 🇳🇦, 🇸🇨, 🇹🇦, 🇺🇿, 🇦🇮,
Nearest to 🎁: gift, birthday, 🎂, christmas, 💝, 🎄, gifts, xmas,
Nearest to 🇪🇸: 🇮🇹, 🇫🇷, 🇷🇺, 🇩🇪, 🇺🇸, 🇯🇵, 🇰🇷, 🇨🇳,
Nearest to ↖️: ↙️, ↗️, ↘️, ⤵️, ⤴️, ↔️, diagonally, diagonal,
Nearest to 🐶: puppy, dog, 🐕, 🐾, 🐱, pup, dogs, pooch,
Nearest to 😀: ☺️, 😃, 😊, 😄, 😸, 🙂, 🌞, 😁,
Nearest to ✅: ✔️, ⭕, 🔰, ❣️, ☑️, ❔, 🔛, 🍡,
Nearest to 🇸🇦: 🇧🇭, 🇶🇦, 🇪🇹, 🇪🇬, 🇾🇪, 🇷🇸, 🇳🇵, 🇲🇦,
Nearest to ⏪: ⏫, ⏬, ⏩, ▶️, ◀️, 🔻, 🔺, rewind,
Nearest to 👨‍🍳: 👩‍🍳, chef, 👨, cooks, chefs, 👨‍💻, sous_chef, waiter,
Nearest to 🎬: film, slate, 📽️, board, directors, 🎥, movie, 🎞️,
Nearest to 📞: phone, 📴, 📱, telephone, handset, ☎️, cellphone, cell_phone,
Nearest to 👨‍💼: 👩‍💼, 👨‍🏭, businessman, 👷‍♂️, 👩‍🏭, worker, 👷‍♀️, 👨‍🌾,
Nearest to 📄: printed, page, pages, facing, print, pasted, printing, webpage,
Nearest to 👢: boots, 👚, 👒, knee, 👠, 👡, thigh, ankle,
Neare

Nearest to 💏: 👨‍👩‍👧‍👦, 👨‍👩‍👧‍👧, 👩‍👩‍👧‍👦, 👨‍👨‍👧‍👦, 👨‍👩‍👦‍👦, 👨‍👩‍👧, girl, 👧,
Nearest to 🌈: 🏳️‍🌈, gay, pride, lesbian, gay_lesbian, queer, lgbt, lesbians,
Nearest to 👏: clap, clapping, applause, claps, clapped, cheers, 👐, 🤝,
Nearest to 🌿: crop, 🌾, plant, plants, crops, 🍅, tomato, grower,
Nearest to 🏕️: campsite, camping, ⛺, campground, campsites, campgrounds, campers, campfire,
Nearest to 🛌: accommodation, sleeping, 🏨, accommodations, 😴, lodgings, hostel, slept,
Nearest to 🦀: crab, cancer, ♋, breast_cancer, red, lobster, colon_cancer, crabs,
Nearest to 🏊‍♀️: 🏊‍♂️, 🏊, swimming, swimmer, 🚣‍♀️, 👱‍♀️, woman, 🤦‍♀️,
Nearest to 🇲🇩: 🇸🇭, 🇳🇮, 🇲🇪, 🇳🇦, 🇸🇨, 🇹🇦, 🇺🇿, 🇦🇮,
Nearest to 🤞: luck, fingers, crossed, 🖐️, 👌, ⚔️, good, ❌,
Nearest to 🇫🇲: 🇸🇭, 🇳🇮, 🇲🇪, 🇳🇦, 🇸🇨, 🇹🇦, 🇺🇿, 🇦🇮,
Nearest to 🤐: lips, 😽, 😦, 😺, sealed, 😖, 😮, 😰,
Nearest to 🇭🇰: 🇲🇴, 🇱🇦, 🇹🇭, 🇸🇲, 🇵🇷, 🇸🇬, 🇩🇬, 🇲🇱,
Nearest to 🌛: 🌜, 🌓, 🌗, 🌝, moon, quarter, 😐, 🌚,
Nearest to ♎: ♍, ♏, ♌, 👲, 🤶, 🇭🇰, haha, 🎅,
Nearest to 🍠: potato, roasted, sweet_potato, sweet_

Nearest to 🛳️: passenger, ship, 🚢, passengers, freighter, ships, vessel, cargo,
Nearest to 👾: alien, invader, monster, 👽, aliens, monsters, 👹, invaders,
Nearest to 🇰🇵: 🇿🇦, 🇰🇷, 🇻🇳, 🇸🇸, 🇬🇪, 🇦🇸, 🇪🇹, 🇿🇼,
Nearest to 🦎: lizard, lizards, frog, reptile, 🕷️, spider, 🐢, snake,
Nearest to 👨‍👩‍👧‍👧: 👨‍👩‍👧‍👦, 👨‍👩‍👦‍👦, 👨‍👩‍👧, 👨‍👩‍👦, 👩‍👩‍👧‍👦, 👩‍👧‍👦, 👩‍👩‍👦‍👦, 👨‍👨‍👧‍👦,
Nearest to 🐲: 🐵, 🐷, 🐉, 🐴, 🦄, face, 🙆, head,
Nearest to 👩‍👩‍👦: 👩‍👩‍👧, 👨‍👨‍👧, 👩‍👩‍👦‍👦, 👩‍👩‍👧‍👦, 👨‍👨‍👦, 👩‍👩‍👧‍👧, 👨‍👩‍👧, 👨‍👩‍👦,
Nearest to ➕: ➖, ➗, heavy, sign, ✔️, plus, ✅, 🛑,
Nearest to ®️: ®️, registering, register, logged, registrations, unregistered, registration, registers,
Nearest to 🍩: 🍩, donuts, bagel, sandwich, cupcake, 🍪, 🍔, krispy_kreme,
Nearest to 🌘: 🌖, 🌒, 🌙, 🌕, 🌔, 🌗, 🌓, 🌑,
Nearest to 🛀: bath, bathing, baths, 🛁, ♨️, bathe, shower, hot,
Nearest to 🙂: 😸, 😀, 😊, ☺️, 😃, 😄, 🙁, 😁,
Nearest to ⏬: ⏫, ◀️, ⏩, ⏪, 🔻, 🔺, 🔼, ▶️,
Nearest to 😌: relieved, pleased, 😥, 😞, thrilled, happy, glad, delighted,
Nearest to 🇻🇮: 🇻🇬, 🇫🇴, 🇨🇰, 🇵🇳, 🇫🇰, 🇸🇧, 🇰🇾, 🇨🇵

Nearest to 🤴: princes, 👑, monarch, prince_charles, throne, prince, prince_harry, crown_prince,
Nearest to 🇲🇶: 🇲🇳, 🇦🇿, 🇱🇻, 🇬🇵, 🇬🇩, 🇬🇾, 🇧🇫, 🇳🇦,
Nearest to 🔼: 🔽, 🔻, 🔺, ◀️, ⏬, ⏫, 🔴, ▶️,
Nearest to ❣️: ❗, 🔛, ❕, ‼️, ⁉️, ❔, 🆙, ❓,
Nearest to 🌇: buildings, 🌆, 🏢, 🏘️, city, sunset, 🌅, downtown,
Nearest to 🚵‍♀️: 🚵‍♂️, 🚵, 🚴‍♀️, 🚴‍♂️, biking, ⛰️, 🚴, mountain_biking,
Nearest to 👷: 👷‍♂️, 🤠, 👷‍♀️, helmet, hat, ⛑️, worker, construction,
Nearest to ⛴️: ⛴️, ferries, ferry_terminal, boat, 🛳️, 🚢, cruise_ship, steamer,
Nearest to 🔕: 🔇, 🔔, notifications, 📳, 🎐, disabled, cancellation, 📟,
Nearest to 📤: tray, 📥, puncher, trays, fridge, darchinyan, dirrell, maidana,
Nearest to 🍰: cake, 🍓, 🍮, strawberry, dessert, cakes, cheesecake, 🍥,
Nearest to 🇪🇭: 🇹🇱, 🇾🇪, 🇸🇸, 🇮🇸, 🇬🇱, 🇲🇵, 🇺🇬, 🇲🇳,
Nearest to 🙆: 🙅, 🐵, 😝, 😻, 😁, 🐲, 🐷, 🤕,
Nearest to 👩‍👦‍👦: 👩‍👦, 👩‍👧‍👦, 👩‍👧, 👩‍👧‍👧, 👨‍👧, 👨‍👧‍👧, 👨‍👧‍👦, 👨‍👦‍👦,
Nearest to 👨‍👩‍👦‍👦: 👨‍👩‍👧‍👦, 👨‍👩‍👧‍👧, 👨‍👩‍👦, 👨‍👩‍👧, 👨‍👨‍👧‍👦, 👨‍👧‍👦, 👩‍👩‍👧‍👦, 👩‍👧‍👦,
Nearest to 🐻: bear, 🐯, 🐵, 😿, 🦊, 🐼, 🐹, 🦁,
Neares

...

Nearest to 🇹🇿: 🇸🇭, 🇳🇮, 🇲🇪, 🇳🇦, 🇸🇨, 🇹🇦, 🇺🇿, 🇦🇮,
Nearest to 🖲️: touchpad, qwerty_keyboard, keypad, stylus, keyboard, ⌨️, touchscreen, qwerty,
Nearest to 🌼: 🌸, 🌻, flower, blossom, 💮, 🌹, blossoms, 🥀,
Nearest to 🇱🇷: 🇸🇭, 🇳🇮, 🇲🇪, 🇳🇦, 🇸🇨, 🇹🇦, 🇺🇿, 🇦🇮,
Nearest to 🚰: 🚱, thirsty, water, thirst, 💦, potable_water, 🐃, tap,
Nearest to 🍉: 🍉, 🍈, melon, watermelons, 🍓, pumpkin, cantaloupe, strawberry,
Nearest to 8️⃣: 7️⃣, 6️⃣, 5️⃣, 4️⃣, 9️⃣, 3️⃣, 2️⃣, 1️⃣,
Nearest to 🐬: whale, 🐋, whales, shark, orca, sea_turtle, turtle, killer_whale,
Nearest to 🔭: telescope, telescopes, astronomy, planetarium, astronomers, observatories, hubble, astronomer,
Nearest to 😓: 😅, 😰, 😥, hard, sweat, 😟, 😹, 😞,
Nearest to 📜: parchment, scroll, scrolls, degree, embossed, engraving, 🏺, inscribed,
Nearest to 😿: 🙀, 😻, 🐱, 😾, 😢, 😹, 😸, 🐶,
Nearest to 🔑: gold, silver, key, 💛, 🥇, precious_metal, precious_metals, 🥈,
Nearest to ❌: ❎, cross, ☦️, wrong, crosses, 🔛, crossed, ⛑️,
Nearest to 🤸‍♂️: 🤸‍♀️, man, 🤦‍♂️, 👱‍♂️, 🚶‍♂️, ⛹️‍♂️, 💏, 👨‍👩‍👦‍👦,
Ne

Nearest to 📰: newspaper, newspapers, 🗞️, broadsheet, dailies, herald, tabloid, gazette,
Nearest to 🇹🇲: 🇸🇭, 🇳🇮, 🇲🇪, 🇳🇦, 🇸🇨, 🇹🇦, 🇺🇿, 🇦🇮,
Nearest to 👡: sandal, sandals, footwear, shoes, 👖, 👚, shoe, 👞,
Nearest to 🚊: 🚊, 🚋, trams, 🚎, 🚞, trolley, 🚡, streetcar,
Nearest to 🇱🇺: 🇸🇭, 🇳🇮, 🇲🇪, 🇳🇦, 🇸🇨, 🇹🇦, 🇺🇿, 🇦🇮,
Nearest to 🇧🇹: 🇸🇭, 🇳🇮, 🇲🇪, 🇳🇦, 🇸🇨, 🇹🇦, 🇺🇿, 🇦🇮,
Nearest to 🚧: yellow, striped, striping, roadwork, bright_yellow, bright_orange, ⭐, red,
Nearest to 🍦: 🍨, soft, cream, 🍡, 🍧, ice, creamy, 🍥,
Nearest to 📚: stack, pile, stacks, piles, books, heap, stacked, piled,
Nearest to 🇹🇴: 🇳🇵, 🇨🇮, 🇲🇳, 🇹🇿, 🇧🇹, 🇦🇱, 🇬🇼, 🇳🇷,
Nearest to 🦆: 🦆, 😗, 🍗, goose, bird, 🐦, 🐔, rabbit,
Nearest to 🇨🇼: 🇲🇸, 🇧🇬, 🇧🇧, 🇱🇸, 🇬🇩, 🇦🇩, 🇵🇼, 🇧🇮,
Nearest to 🇮🇱: 🇱🇧, 🇸🇾, 🇵🇸, 🇮🇷, 🇪🇹, 🇪🇬, 🇸🇦, 🇿🇼,
Nearest to 🇫🇴: 🇻🇮, 🇨🇰, 🇵🇳, 🇫🇰, 🇸🇧, 🇧🇻, 🇨🇵, 🇰🇾,
Nearest to 🇸🇭: 🇸🇭, 🇳🇮, 🇲🇪, 🇳🇦, 🇸🇨, 🇹🇦, 🇺🇿, 🇦🇮,
Nearest to 2️⃣: 3️⃣, 4️⃣, 5️⃣, 6️⃣, 8️⃣, 7️⃣, 9️⃣, 1️⃣,
Nearest to 🍟: fries, french_fries, 🇵🇫, 🍔, 🇬🇫, burger, burgers, cheeseburger,
Nearest to 🐴: 🐷, 🐵, 🐲,

Nearest to 🐥: chick, 🐤, 🐣, baby, 👶, chicks, newborn, babies,
Nearest to 🐟: freshwater, fish, 🐠, trout, sturgeon, mussels, striped_bass, saltwater,
Nearest to 📓: 📘, black, white, notebook, 📙, ✒️, 📔, ⭐,
Nearest to 🇩🇲: 🇸🇭, 🇳🇮, 🇲🇪, 🇳🇦, 🇸🇨, 🇹🇦, 🇺🇿, 🇦🇮,
Nearest to 🥖: baguette, 🍞, bread, loaf, 🥐, pastry, 🍝, polenta,
Nearest to 😝: 😜, 😛, 👅, 😻, 😍, 🙄, 😑, 😁,
Nearest to 🗑️: trash, garbage, rubbish, bin, recycling_bin, waste, bins, dumpster,
Nearest to 🔚: 🔙, 🔜, 🔝, 📩, ⤴️, ⬅️, 🔛, ⤵️,
Nearest to 👩‍🌾: 👨‍🌾, farmer, 🤦‍♀️, 👱‍♀️, woman, 👨‍👩‍👧‍👧, 👨‍👩‍👦‍👦, 👨‍👩‍👧‍👦,
Nearest to ♏: ♉, ♎, 🤶, ♍, 🎅, 🐐, 🐪, ya,
Nearest to 🌊: ocean, sea, waves, beach, seas, wave, shoreline, lagoon,
Nearest to 👯‍♂️: 👯‍♀️, 👯, 😽, 😙, 🐰, 🙉, 😻, 😈,
Nearest to 😲: stunned, shocked, astonished, dumbfounded, 😯, 😨, horrified, 😕,
Nearest to 🇦🇫: 🇮🇶, 🇪🇹, 🇽🇰, 🇦🇸, 🇾🇪, 🇳🇵, 🇷🇸, 🇮🇷,
Nearest to 🚼: baby, nursery, 👶, 🚯, 🐥, 🐤, 🍼, symbol,
Nearest to 🏅: medal, medals, gold_medal, 🥇, 🥈, 🥉, gold_medals, sports,
Nearest to 🗻: ⛰️, 🚠, mountains, 🏔️, mounted, mounta

Nearest to 🚭: 🚬, symbol, 🕉️, smoking, 🚯, 🔣, 🚱, ☮️,
Nearest to 🚞: 🚡, railway, 🚂, tram, 🚊, 🚠, train, 🚄,
Nearest to 🇨🇲: 🇸🇭, 🇳🇮, 🇲🇪, 🇳🇦, 🇸🇨, 🇹🇦, 🇺🇿, 🇦🇮,
Nearest to 🇮🇨: 🇵🇳, 🇫🇰, 🇰🇾, 🇸🇧, 🇲🇵, 🇧🇻, 🇨🇵, 🇳🇫,
Nearest to 🇦🇨: 🇨🇵, 🇧🇻, 🇫🇰, 🇵🇳, 🇳🇫, 🇸🇧, 🇦🇽, 🇰🇾,
Nearest to 🔶: 🔸, 🔷, 🔹, ♦️, diamonds, 🔼, ⬛, 🔴,
Nearest to 🌧️: ⛈️, 🌦️, 🌨️, 🌩️, rain, cloud, ☁️, ⛅,
Nearest to 🖇️: linked, rubber_bands, 🔗, linking, attached, coded, ✂️, interlinked,
Nearest to 🇱🇨: 🇲🇨, 🇪🇷, 🇪🇪, 🇧🇴, 🇼🇸, 🇩🇲, 🇨🇲, 🇱🇻,
Nearest to ☘️: 🍀, purple, 🍡, 🌻, emerald_isle, irish, 🇨🇽, 🎀,
Nearest to 🏺: vase, jar, ⚱️, vases, jug, jars, teapot, urn,
Nearest to 🈚: 🈶, 🈵, 🈺, 🈷️, 🈳, 🈴, 🈲, unified,
Nearest to 🐦: bird, birds, 🐤, owl, 🦉, squirrel, bald_eagle, avian,
Nearest to 🆕: 🆚, squared, 🆓, 🆒, 🈶, 🈺, 🈵, 🆗,
Nearest to 💨: steam, ♨️, 💫, 💥, 🚭, 🔰, wind, 💦,
Nearest to 🏜️: desert, 🏝️, 🌵, deserts, arid, mojave_desert, sand_dunes, desolate,
Nearest to 🇧🇾: 🇸🇭, 🇳🇮, 🇲🇪, 🇳🇦, 🇸🇨, 🇹🇦, 🇺🇿, 🇦🇮,
Nearest to 🇹🇭: 🇮🇩, 🇸🇬, 🇻🇳, 🇲🇾, 🇪🇹, 🇭🇺, 🇳🇵, 🇸🇦,
Nearest to 🕳️: 🕳️, holes, ⛳, fai

Nearest to 🌌: universe, sky, planets, ☄️, galaxies, 🌃, milky_way, galactic,
Nearest to 🇧🇳: 🇸🇭, 🇳🇮, 🇲🇪, 🇳🇦, 🇸🇨, 🇹🇦, 🇺🇿, 🇦🇮,
Nearest to 🗼: tower, red, 🔰, towers, 🏮, 🔻, 🔺, 🖥️,
Nearest to 🍪: chocolate, cookie, biscuit, 🍮, cookies, 🍬, ice_cream, 🍰,
Nearest to 🐸: frog, 🦊, 👺, 🦄, 🐰, 🐵, frogs, 🐯,
Nearest to ⛺: tent, camping, 🏕️, tents, campsite, campsites, 🎪, campground,
Nearest to 🇰🇬: 🇸🇭, 🇳🇮, 🇲🇪, 🇳🇦, 🇸🇨, 🇹🇦, 🇺🇿, 🇦🇮,
Nearest to 😹: 😂, 😸, 😺, 😻, 😭, 😊, 😆, 😄,
Nearest to 🍚: 🍜, cooked, 🍲, steamed, soup, boiled, 🍳, 🍨,
Nearest to 🍗: chicken, 🐔, poultry, meat, chickens, pork, pig, turkeys,
Nearest to ⛷️: skier, ski, 🎿, skiing, skiers, 🏂, snowboarder, snowboarding,
Nearest to 💜: 💙, ❤️, heart, purple, 💛, 🖤, ♥️, 💚,
Nearest to 🇬🇲: 🇸🇭, 🇳🇮, 🇲🇪, 🇳🇦, 🇸🇨, 🇹🇦, 🇺🇿, 🇦🇮,
Nearest to 🙃: sarcasm, 😼, 😜, silly, 😉, 😏, 😛, sarcastic,
Nearest to 🗞️: delivery, 📰, newspaper, 🚚, newspapers, deliveries, dailies, herald,
Nearest to 💵: 💶, 💴, 💷, dollar, 🤑, 🇺🇸, note, 🇷🇺,
Nearest to 🈯: 🈶, 🈵, 🈺, 🈷️, 🈚, finger, 🈴, 🈲,
Nearest to ⚠️: aler